In [ ]:
import cv2
import numpy as np
import tkinter as tk # Used to get screen resolution

def create_fullscreen_kaleidoscope(image_path, output_path, segments=12):
    # --- 1. Get Screen Resolution ---
    root = tk.Tk()
    root.withdraw() # Hide the little tkinter window
    screen_w = root.winfo_screenwidth()
    screen_h = root.winfo_screenheight()
    print(f"Detected Screen Size: {screen_w}x{screen_h}")

    # --- 2. Load and Prepare Image ---
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found.")
        return

    # To fill a rectangular screen with a radial pattern without black bars, 
    # we need to generate a square image as big as the screen's largest dimension,
    # then crop it back down to the screen shape.
    target_dim = max(screen_w, screen_h)
    
    # Resize the input image so it is at least as big as the target_dim
    h, w = img.shape[:2]
    scale = target_dim / min(h, w)
    
    # We resize using cubic interpolation for better quality when scaling up
    new_w = int(w * scale)
    new_h = int(h * scale)
    img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    
    # Crop to a square (target_dim x target_dim) from the center
    start_x = (new_w - target_dim) // 2
    start_y = (new_h - target_dim) // 2
    img = img[start_y:start_y+target_dim, start_x:start_x+target_dim]
    
    # Update dimensions (now it is perfectly square)
    h, w = img.shape[:2]
    center = (w // 2, h // 2)

    # --- 3. Create the Wedge Mask ---
    angle = 360 / segments
    mask = np.zeros_like(img)
    
    # Calculate triangle vertices
    theta = (angle / 2) * (np.pi / 180)
    radius = int(np.sqrt((w/2)**2 + (h/2)**2)) 
    
    x_offset = int(radius * np.sin(theta))
    y_offset = int(radius * np.cos(theta))
    
    pt1 = center
    pt2 = (center[0] - x_offset, center[1] - y_offset)
    pt3 = (center[0] + x_offset, center[1] - y_offset)
    
    triangle_cnt = np.array([pt1, pt2, pt3], np.int32)
    cv2.fillPoly(mask, [triangle_cnt], (255, 255, 255))
    
    masked_img = cv2.bitwise_and(img, mask)

    # --- 4. Generate Kaleidoscope ---
    # We generate the large square pattern first
    square_canvas = np.zeros_like(img)
    
    for i in range(segments):
        current_angle = i * angle
        M = cv2.getRotationMatrix2D(center, current_angle, 1.0)
        
        if i % 2 == 1:
            flipped_wedge = cv2.flip(masked_img, 1)
            rotated_wedge = cv2.warpAffine(flipped_wedge, M, (w, h))
        else:
            rotated_wedge = cv2.warpAffine(masked_img, M, (w, h))
            
        square_canvas = np.maximum(square_canvas, rotated_wedge)

    # --- 5. Crop to Monitor Size ---
    # We now have a massive square. We need to cut out the center 
    # to match your specific screen_w and screen_h.
    
    final_img = np.zeros((screen_h, screen_w, 3), dtype=np.uint8)
    
    # Calculate crop coordinates
    crop_y = (target_dim - screen_h) // 2
    crop_x = (target_dim - screen_w) // 2
    
    final_img = square_canvas[crop_y:crop_y+screen_h, crop_x:crop_x+screen_w]

    # --- 6. Save and Display Fullscreen ---
    cv2.imwrite(output_path, final_img)
    print(f"Saved {output_path} ({screen_w}x{screen_h})")
    
    #window_name = "Kaleidoscope Fullscreen"
    #cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    #cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    #cv2.imshow(window_name, final_img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

if __name__ == "__main__":
    # Remember to change 'citylights1.jpg' to your actual file
    create_fullscreen_kaleidoscope('citylights1.jpg', 'output.jpg', segments=12)

Detected Screen Size: 1920x1080
Saved output.jpg (1920x1080)
